# Introduction to Ray for Distributed Computing in Python

## Overview

Ray is an open-source framework for scaling Python applications. It provides a simple, universal API for building distributed applications. Ray is particularly useful in machine learning and artificial intelligence workflows, where it can help parallelize computations and manage distributed resources efficiently.

In this lecture, we'll cover:

1. Basic Ray concepts
2. Setting up Ray
3. Remote functions and parallel execution
4. Ray Tasks vs. Actors
5. Shared memory and object stores
6. Ray for Machine Learning
7. Best practices and considerations

Let's begin by installing Ray and importing necessary libraries.

In [1]:
!pip install ray

import ray
import numpy as np
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 MB 1.5 MB/s eta 0:00:0000:0100:020m


## 1. Basic Ray Concepts

Ray's core abstraction is a task - a remote function invocation. Ray uses these tasks to distribute computation across a cluster of machines. The key components of Ray are:

- **Workers**: Python processes that execute tasks.
- **Drivers**: The main Python script that defines and invokes remote tasks.
- **Object Store**: A distributed shared-memory object store.
- **Scheduler**: Assigns tasks to workers.

## 2. Setting up Ray

To use Ray, we first need to initialize it. This can be done locally or on a cluster.

In [2]:
# Initialize Ray
ray.init()

# You can also specify resources
# ray.init(num_cpus=4, num_gpus=1)

2025-11-06 23:40:38,301	INFO worker.py:1951 -- Started a local Ray instance.


Python version:,3.13.5
Ray version:,2.49.2


## 3. Remote Functions and Parallel Execution

Ray allows you to execute functions remotely using the `@ray.remote` decorator. These functions can run in parallel on different machines or cores.

In [3]:
@ray.remote
def slow_function(i):
    time.sleep(1)  # Simulate a slow operation
    return i * i

# Execute functions in parallel
start_time = time.time()
results = ray.get([slow_function.remote(i) for i in range(4)])
end_time = time.time()

print(f"Results: {results}")
print(f"Time taken: {end_time - start_time:.2f} seconds")

Results: [0, 1, 4, 9]
Time taken: 2.74 seconds


In this example, we define a `slow_function` that simulates a time-consuming operation. By using `@ray.remote`, we can execute multiple instances of this function in parallel, significantly reducing the total execution time.

## 4. Ray Tasks vs. Actors

Ray provides two main abstractions for parallel computation:

1. **Tasks**: Stateless functions (like we saw above)
2. **Actors**: Stateful workers

Let's look at an example using an Actor:

In [4]:
@ray.remote
class Counter:
    def __init__(self):
        self.value = 0
    
    def increment(self):
        self.value += 1
        return self.value

# Create an actor
counter = Counter.remote()

# Increment the counter in parallel
results = ray.get([counter.increment.remote() for _ in range(5)])
print(f"Counter values: {results}")

Counter values: [1, 2, 3, 4, 5]


In this example, we create a `Counter` actor that maintains its state across method calls. This is useful for scenarios where you need to maintain state in a distributed setting.

## 5. Shared Memory and Object Stores

Ray uses a distributed object store to efficiently pass large objects between tasks. This is particularly useful for machine learning workloads with large datasets or models.

In [5]:
# Create a large object
large_matrix = np.random.rand(1000, 1000)

# Put the object in the object store
matrix_id = ray.put(large_matrix)

@ray.remote
def matrix_sum(matrix):
    return np.sum(matrix)

# Use the object reference in a task
result = ray.get(matrix_sum.remote(matrix_id))
print(f"Sum of matrix elements: {result}")

Sum of matrix elements: 499805.733176693


By using `ray.put()`, we can efficiently share large objects between tasks without the need for serialization and deserialization.

## 6. Ray for Machine Learning

Ray provides several libraries specifically designed for machine learning workloads:

- **Ray Tune**: For hyperparameter tuning
- **Ray Train**: For distributed model training
- **Ray Serve**: For model serving

Let's look at a simple example using Ray Tune for hyperparameter optimization:

In [9]:
!pip install "ray[tune]"
from ray import tune
from ray.tune.schedulers import ASHAScheduler

def objective(config):
    # Simulate a model training process
    for step in range(100):
        intermediate_score = config["a"] * step + config["b"]
        tune.report(score=intermediate_score)

analysis = tune.run(
    objective,
    config={
        "a": tune.uniform(0, 1),
        "b": tune.uniform(0, 20)
    },
    num_samples=10,
    scheduler=ASHAScheduler(metric="score", mode="max")
)

print("Best config:", analysis.get_best_config(metric="score", mode="max"))

2025-11-06 23:54:01,618	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2025-11-06 23:54:08,569	ERROR tune_controller.py:1331 -- Trial task failed for trial objective_ec937_00005
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/_private/worker.py", line 2882, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/_private/worker.py", line 968, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions

Trial name
objective_ec937_00000
objective_ec937_00001
objective_ec937_00002
objective_ec937_00003
objective_ec937_00004
objective_ec937_00005
objective_ec937_00006
objective_ec937_00007
objective_ec937_00008
objective_ec937_00009


2025-11-06 23:54:08,623	ERROR tune_controller.py:1331 -- Trial task failed for trial objective_ec937_00004
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/_private/worker.py", line 2882, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/ray/_private/worker.py", line 968, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions

TuneError: ('Trials did not complete', [objective_ec937_00000, objective_ec937_00001, objective_ec937_00002, objective_ec937_00003, objective_ec937_00004, objective_ec937_00005, objective_ec937_00006, objective_ec937_00007, objective_ec937_00008, objective_ec937_00009])

This example demonstrates how Ray Tune can be used to optimize hyperparameters in a distributed manner.

## 7. Best Practices and Considerations

When using Ray, keep the following best practices in mind:

1. **Task Granularity**: Ensure tasks are not too small (overhead of distribution) or too large (limits parallelism).
2. **Resource Management**: Specify CPU and GPU requirements for tasks and actors when necessary.
3. **Error Handling**: Use Ray's built-in retry mechanisms for fault tolerance.
4. **Monitoring**: Utilize Ray's dashboard for cluster monitoring and debugging.
5. **Data Transfer**: Minimize data transfer between nodes by using Ray's object store effectively.

## Conclusion

Ray provides a powerful framework for distributed computing in Python, with particular strengths in machine learning workflows. Its simple API allows for easy parallelization of existing code, while its specialized libraries offer advanced functionality for ML tasks.

In an MLOps context, Ray can be particularly useful for:
- Distributed data preprocessing
- Parallel model training
- Hyperparameter tuning at scale
- Distributed inference

By integrating Ray into your MLOps pipeline, you can significantly improve the scalability and efficiency of your machine learning workflows.